In [1]:
from importlib import reload
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [189]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    print('reload failed')
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    print('reload failed')
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *

In [135]:
reload(dataMethods)
from modules.dataMethods import *

In [175]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)
# pm2 = note_bin2pm(noteB2)
# pm2.write('midi/basic_test_tested.mid')

[[27, 0, 0, 3, 19, 27], [0, 0, 0, 3, 20, 28], [5, 3, 35, 3, 14, 25], [32, 0, 0, 3, 16, 26], [2, 3, 27, 0, 21, 17], [34, 0, 0, 1, 13, 19], [3, 0, 42, 0, 20, 11], [5, 0, 38, 0, 21, 11], [32, 0, 3, 1, 20, 13], [2, 0, 42, 0, 28, 9], [7, 0, 59, 2, 16, 8], [31, 0, 2, 2, 15, 5]]


In [75]:
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro['midi_filename'])


In [190]:
data_path = 'C:/Users\Andrew/Documents/mlprojects/Datasets/MaestroV2.00/maestro-v2.0.0/'
X, Y, X_test, Y_test, n_notes = get_processed_note_bin_data(data_path)

################# new example! Of length 4197
duration_exceeded:  7
shifts_exceeded:  0
################# new example! Of length 6072
duration_exceeded:  7
shifts_exceeded:  0
################# new example! Of length 2473
duration_exceeded:  7
shifts_exceeded:  0
################# new example! Of length 3900
duration_exceeded:  7
shifts_exceeded:  0
################# new example! Of length 11581
exceeded: 11
exceeded: 21
exceeded: 36
exceeded: 31
exceeded: 28
exceeded: 29
exceeded: 24
duration_exceeded:  14
shifts_exceeded:  7


In [191]:
print(len(Y))
note_bin2pm(Y[0]).write('midi/120.mid')
note_bin2pm(Y[50]).write('midi/1250.mid')
note_bin2pm(Y[100]).write('midi/12100.mid')
note_bin2pm(Y[150]).write('midi/12150.mid')

197


In [194]:
pm = pretty_midi.PrettyMIDI('midi/BergOriginal.midi')
desus(pm)
note_bin2pm(pm2note_bin(pm)).write('midi/BergNoPedal.mid')